In [27]:
!pip install supervision

In [28]:
!pip install ultralytics

In [29]:
from ultralytics import YOLO

In [30]:
model = YOLO('./models/yolov8n.pt')

In [31]:
print(model.names)

{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microw

In [32]:
import numpy as np
import supervision as sv
from ultralytics import YOLO
import cv2
from tqdm import tqdm

In [33]:
model = YOLO('./models/yolov8n.pt')

In [34]:
selected_classes = [2,3]

In [35]:
SOURCE_VIDEO_PATH = './source.mp4'
TARGET_VIDEO_PATH = './result.mp4'

In [36]:
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

In [37]:
video_info

VideoInfo(width=1920, height=1080, fps=30, total_frames=9184)

In [38]:
generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)

In [39]:
generator

<generator object get_video_frames_generator at 0x7ecac93cee90>

In [40]:
tracker = sv.ByteTrack(track_activation_threshold=0.25, lost_track_buffer=60, minimum_matching_threshold=0.8, frame_rate=30)

In [41]:
tracker

In [42]:
box_annotator = sv.BoundingBoxAnnotator()

SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.


In [43]:
box_annotator

In [44]:
dot_annotater = sv.DotAnnotator(radius=0)

In [45]:
LINE_START_ONE = sv.Point(250, 225)
LINE_END_ONE = sv.Point(0, 175)

In [46]:
LINE_START_TWO = sv.Point(375, 225)
LINE_END_TWO = sv.Point(270, 225)

In [47]:
LINE_START_THREE = sv.Point(600, 280)
LINE_END_THREE = sv.Point(425, 230)

In [48]:
line_zones = [
    sv.LineZone(start=LINE_START_ONE, end=LINE_END_ONE),
    sv.LineZone(start=LINE_START_TWO, end=LINE_END_TWO),
]

In [49]:
line_zone_annotator = sv.LineZoneAnnotator(
    thickness=1,
    text_thickness=1
)

In [50]:
def filter_vehicle_detections(detections):
  vehicle_detections = detections[np.isin(detections.class_id, selected_classes)]
  return vehicle_detections

In [51]:
def callback(frame: np.ndarray, _:int) -> np.ndarray:
  results = model(frame)[0]
  detections = sv.Detections.from_ultralytics(results)
  detections = filter_vehicle_detections(detections)
  detections = tracker.update_with_detections(detections)

  annotated_frame = box_annotator.annotate(scene=frame.copy(), detections=detections)

  annotated_frame = dot_annotater.annotate(scene=annotated_frame, detections=detections)

  for line_zone in line_zones:
    line_zone.trigger(detections=detections)

  for line_zone in line_zones:
    annotated_frame = line_zone_annotator.annotate(frame=annotated_frame, line_counter=line_zone)

  return annotated_frame


In [ ]:
def process_video_with_progress(source_path: str, target_path: str, callback, pregress_desc: str="Processing"):
  cap = cv2.VideoCapture(source_path)
  total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

  with tqdm(total=total_frames, desc=pregress_desc) as pbar:
    def wrapped_callback(frame: np.ndarray, frame_idx: int) -> np.ndarray:
      result_frame = callback(frame, frame_idx)
      pbar.update(1)
      return result_frame
    sv.process_video(
        source_path=source_path,
        target_path=target_path,
        callback=wrapped_callback,
        # show_progress=True
    )

process_video_with_progress(
      source_path=SOURCE_VIDEO_PATH,
      target_path=TARGET_VIDEO_PATH,
      callback=callback
  )

Processing:   0%|          | 0/9184 [00:00<?, ?it/s]


0: 384x640 4 persons, 5 cars, 1 bus, 98.2ms
Speed: 2.5ms preprocess, 98.2ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)


Processing:   0%|          | 1/9184 [00:00<51:07,  2.99it/s]


0: 384x640 4 persons, 6 cars, 1 bus, 1 umbrella, 80.0ms
Speed: 2.4ms preprocess, 80.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


Processing:   0%|          | 2/9184 [00:00<31:15,  4.89it/s]

...


0: 384x640 1 person, 5 cars, 2 buss, 36.4ms
Speed: 2.1ms preprocess, 36.4ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)


Processing:   8%|▊         | 726/9184 [01:25<15:47,  8.93it/s]


0: 384x640 2 persons, 6 cars, 1 bus, 36.5ms
Speed: 2.0ms preprocess, 36.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


Processing:   8%|▊         | 727/9184 [01:25<17:21,  8.12it/s]


0: 384x640 1 person, 6 cars, 2 buss, 1 truck, 36.4ms
Speed: 2.0ms preprocess, 36.4ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
